In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('https://raw.githubusercontent.com/olexandryermilov/deeploma/master/datasets/messages.csv')
data.head

<bound method NDFrame.head of                      message_text message_type
0                  Hi, I am Sasha         fact
1   Where is the nearest library?     question
2                         I am 21         fact
3            She works in library         fact
4        it is quite unbeliavable        trash
..                            ...          ...
85          nice to meet you too!         fact
86                  bad day today         fact
87                             Hi         fact
88           I found 5 cats today         fact
89     How many cats I saw today?     question

[90 rows x 2 columns]>

In [2]:
X_train = data['message_text'].values
Y_train = data['message_type'].values
Y_train

array(['fact', 'question', 'fact', 'fact', 'trash', 'request', 'request',
       'question', 'fact', 'trash', 'trash', 'fact', 'trash', 'fact',
       'fact', 'question', 'fact', 'fact', 'question', 'question',
       'question', 'trash', 'request', 'request', 'request', 'request',
       'request', 'request', 'request', 'request', 'request', 'request',
       'request', 'request', 'request', 'request', 'request', 'request',
       'request', 'request', 'request', 'question', 'trash', 'trash',
       'fact', 'fact', 'question', 'request', 'trash', 'trash', 'fact',
       'fact', 'fact', 'fact', 'fact', 'question', 'question', 'fact',
       'question', 'question', 'fact', 'fact', 'fact', 'fact', 'question',
       'fact', 'question', 'question', 'fact', 'question', 'question',
       'fact', 'fact', 'fact', 'fact', 'fact', 'fact', 'fact', 'fact',
       'fact', 'fact', 'fact', 'fact', 'fact', 'fact', 'fact', 'fact',
       'fact', 'fact', 'question'], dtype=object)

In [21]:
import json
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oleksandry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
#!pip3 install nltk

Looking in indexes: https://repo.dev.wixpress.com/artifactory/api/pypi/pypi-local/simple, https://pypi.python.org/simple
     |████████████████████████████████| 1.4MB 580kB/s eta 0:00:01
  Created wheel for nltk: filename=nltk-3.5-cp37-none-any.whl size=1434674 sha256=5de9c6807f55d4e8dcc0cd4f7647029571db2a4ed54657ce1607d740cda3f3cc
  Stored in directory: /Users/oleksandry/Library/Caches/pip/wheels/ae/8c/3f/b1fe0ba04555b08b57ab52ab7f86023639a526d8bc8d384306
Successfully built nltk


In [6]:
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def text_prepare(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower()# lowercase text
    text = re.sub(REPLACE_BY_SPACE_RE," ", text)# replace REPLACE_BY_SPACE_RE symbols by space in text
    text = re.sub(BAD_SYMBOLS_RE,"", text)# delete symbols which are in BAD_SYMBOLS_RE from text
    text = re.sub('  ', ' ', text)
    text = re.sub('  ', ' ', text)
    #text = ' '.join(list(filter(lambda x: x not in STOPWORDS, text.split(" ")))) # delete stopwords from text
    return text

In [7]:
def my_bag_of_words(text, words_to_index, dict_size):
    """
        text: a string
        dict_size: size of the dictionary
        
        return a vector which is a bag-of-words representation of 'text'
    """
    result_vector = np.zeros(dict_size)
    for x in text_prepare(text).split(" "):
      if(x in words_to_index):
        result_vector.itemset(words_to_index[x], 1)
    return result_vector

In [8]:
def countQuestions(text):
  x = 0
  for char in text:
    if char == '?':
      x = x + 1
  return x    

In [9]:
def count_wh_words(text):
  x = 0
  for word in text_prepare(text).split(' '):
    if(word == 'when' or word == 'who' or word == 'why' or word =='whom' or word == 'where'):
      x = x + 1
  return x     

In [10]:
words = [x for item in X_train for x in text_prepare(item).split(' ') ]
unique, counts = np.unique(words, return_counts=True)
words_counts= dict(zip(unique, counts))
DICT_SIZE = 5000
most_common_words = sorted(words_counts.items(), key=lambda x: x[1], reverse=True)[:DICT_SIZE]
WORDS_TO_INDEX = {j:i for i,j in enumerate(words_counts)}
INDEX_TO_WORDS = {i:j for i,j in enumerate(words_counts)}
ALL_WORDS = WORDS_TO_INDEX.keys()

In [11]:
unique_t, counts_t = np.unique([x for x in Y_train], return_counts=True)
tags_counts = dict(zip(unique_t, counts_t))
tags_counts.keys()

dict_keys(['fact', 'question', 'request', 'trash'])

In [12]:
def prepare_text_for_model(text):
  bow = my_bag_of_words(text, WORDS_TO_INDEX, DICT_SIZE)
  questions = np.array([countQuestions(text)])
  wh_words = np.array([count_wh_words(text)])
  return sp_sparse.csr_matrix(np.concatenate((bow, np.concatenate((questions, wh_words)))))

In [13]:
from scipy import sparse as sp_sparse
X_train_mybag = sp_sparse.vstack([prepare_text_for_model(text) for text in X_train])
X_train_mybag.shape

(90, 5002)

In [14]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer(classes=sorted(tags_counts.keys()))
TAGS_FOR_INDEX = {j:i for i,j in enumerate(tags_counts)}
INDEX_TO_TAGS = {i:j for i,j in enumerate(tags_counts)}
y_train = [TAGS_FOR_INDEX[x] for x in Y_train]

In [33]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics 
x_train, x_test, y_train_, y_test = train_test_split(X_train_mybag, y_train, test_size=0.5)
dtree_model = DecisionTreeClassifier(max_depth = 4).fit(x_train, y_train_) 
y_pred = dtree_model.predict(x_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print(metrics.confusion_matrix(y_test, y_pred))

Accuracy: 0.9111111111111111
[[22  0  0  0]
 [ 2  6  0  0]
 [ 0  0 13  0]
 [ 1  0  1  0]]


In [19]:
INDEX_TO_TAGS[dtree_model.predict(prepare_text_for_model('What is a road?'))[0]]


'question'

In [20]:
#POST /classify_message
req = json.loads(REQUEST)['body']
#print(req)
text = req['text']  
print(json.dumps({'answer': INDEX_TO_TAGS[dtree_model.predict(prepare_text_for_model('What is a road?'))[0]]}))

NameError: name 'REQUEST' is not defined